## Contents:

- **[Import Libraries](#Import-Libraries)**.  
    
- **[Data Cleaning & EDA](#Data-Cleaning-Exploratory-Data-Analysis)**.  

- **[Model Preparation](#Model-Preparation)**. 

 - **[Modeling](#Modeling)**.  
   - **[Baseline Model](#Baseline-Model)**. 
   - **[DBSCAN](#DBSCAN)**.  
   - **[K-Means](#K-Means)**.  
   
- **[Model Evaluation](#Model-Evaluation)**.  

- **[Conclusions and Recommendations](#Conclusions-and-Recommendations)**.  

- **[References](#References)**.

### Importing Libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN , KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split

#from sklearn.datasets import make_blobs
#from sklearn.linear_model import LogisticRegression
#from sklearn.decomposition import PCA

RANDOM_STATE = 7777
%matplotlib inline


### Model Preparation

In [ ]:
X = 
y = 

In [ ]:
#Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = RANDOM_STATE)

In [ ]:
#scale our data 
ss = StandardScaler()

#fit and transform Data
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

### Modeling

#### Baseline Model

In [ ]:
# # instantiate DummyClassifier
# dummy = DummyClassifier(strategy="most_frequent")
# dummy.fit(X_train, y_train)

# # score on test
# print('Test Score:', dummy.score(X_test, y_test))

# # score on train
# print('Train Score:', dummy.score(X_train, y_train))

# # score on cross val
# print('Cross Val Score:', cross_val_score(dummy, X, y, cv =5).mean())

In [ ]:
# # Instantiate PCA & logisticRegression
# pca = PCA(random_state=RANDOM_STATE)
# lr_pca = LogisticRegression(random_state=RANDOM_STATE)

# # Get PC's by applying transformer on data
# Z_train = pca.fit_transform(X_train_ss)
# Z_test  = pca.transform(X_test_ss)

# #fitting model
# lr_pca.fit(Z_train, y_train)

# print('lr_pca Score:',lr_pca.score(Z_test, y_test))

In [ ]:
#scaling 
ss = StandardScaler()
X_scaled = ss.fit_transform(df)

#### DBSCAN

In [ ]:
dbscan = DBSCAN(eps = [.5, .65, .8],
               min_samples= [5, 10, 20]
               ) 

dbscan.fit(X_scaled);

In [ ]:
print("Silhoutte Score : ", silhouette_score(X_scaled, dbscan.labels_))

#### K-Means 

In [ ]:
#Fit a k-means clustering model
km = KMeans(n_clusters= [3,5,10,15], 
            n_init= [10, 20, 30],
            max_iter= [300, 400, 500],
            random_state=2020)
#fitting model
km.fit(X_scaled)

In [ ]:
#sum of squared errors for each cluster.
print("Inertia: ", km.inertia_)

#Silhouette Score
print("Silhouette Score: ", silhouette_score(X_scaled, km.labels_))

### Model Evaluation

### Conclusions and Recommendations

### References